# Hypothesis testing

## Introduction and definitions

In [1]:
#: Import numerical and plotting libraries
import numpy as np
# Print to four digits of precision
np.set_printoptions(precision=4, suppress=True)
import numpy.linalg as npl
import matplotlib.pyplot as plt

This exercise returns to the psychopathy of students from Berkeley and MIT.  It continues from the `on_dummies` exercise.  Make sure you have done that exercise before doing this one.

Here are the psychopathy questionnaire scores from another set of 5 students
from Berkeley and MIT

In [2]:
ucb_psycho = np.array([2.9277, 9.7348, 12.1932, 12.2576, 5.4834])
n_ucb = len(ucb_psycho)
mit_psycho = np.array([7.2937, 11.1465, 13.5204, 15.053, 12.6863])
n_mit = len(mit_psycho)

$\newcommand{\yvec}{\vec{y}}$

The `psychopathy` values will be our `y` vector $\yvec$.

In [3]:
# Give name Y to psychopathy, for reading convenience.
Y = np.concatenate([ucb_psycho, mit_psycho])
# Call the number of observations "n"
n = len(Y)
n

We  use the general linear model to do a two-level (UCB, MIT) single factor
(college) analysis of variance on these data.

Our model is that the Berkeley student data are drawn from some distribution
with a mean value that is characteristic for Berkeley: $y_i = \mu_{Berkeley} +
e_i$ where $i$ corresponds to a student from Berkeley.  There is also a
characteristic but possibly different mean value for MIT: $\mu_{MIT}$:

$$
\newcommand{\xvec}{\vec{x}}
\newcommand{\evec}{\vec{\varepsilon}}
\newcommand{\Xmat}{\boldsymbol X}
\newcommand{\bvec}{\vec{\beta}}
\newcommand{\bhat}{\hat{\bvec}}
\newcommand{\yhat}{\hat{\yvec}}
$$

$$
y_i = \mu_{Berkeley} + e_i  \space\mbox{if}\space 1 \le i \le 5 \\
y_i = \mu_{MIT} + e_i \space\mbox{if}\space 6 \le i \le 10
$$

Here is the design matrix for this ANOVA, with dummy variables corresponding to
the UCB and MIT student groups:

In [4]:
#- Create design matrix for UCB / MIT ANOVA
X = np.zeros((n, 2))
X[:n_ucb, 0] = 1  # UCB indicator
X[n_ucb:, 1] = 1  # MIT indicator
# Show the result
X

The betas are given by:

$$
\bhat = (\Xmat^T \Xmat){-1}\Xmat^T \yvec
$$

In [5]:
#- Calculate transpose of design matrix multiplied by data, and therefore
#- calculate beta vector
B = npl.inv(X.T @ X) @ X.T @ Y
# Show the result
B

## Hypothesis testing with contrasts

Remember the student’s t statistic from the general linear model :

$$
\newcommand{\cvec}{\vec{c}}
$$

$$
t = \frac{\cvec^T \bhat}
{\sqrt{\hat{\sigma}^2 \cvec^T (\Xmat^T \Xmat)^+ \cvec}}
$$

Let’s consider the top half of the t statistic, $c^T \bhat$.

Our hypothesis is that the mean psychopathy score for MIT students,
$\mu_{MIT}$, is higher than the mean psychopathy score for Berkeley students,
$\mu_{Berkeley}$.  What contrast vector $\cvec$ do we need to apply to $\bhat$
to express the difference between these means?  Apply this contrast vector to
$\bhat$ to get the top half of the t statistic.

In [6]:
#- Contrast vector to express difference between UCB and MIT
#- Resulting value will be high and positive when MIT students have
#- higher psychopathy scores than UCB students
c = np.array([-1, 1])
top_of_t = c @ B
# Show the result
top_of_t

In [7]:
assert top_of_t > 0, 'Oops, did you subtract the wrong value?'
assert np.isclose(top_of_t, np.mean(mit_psycho) - np.mean(ucb_psycho))

Now the bottom half of the t statistic.  Remember this is
$\sqrt{\hat{\sigma}^2 \cvec^T (\Xmat^T \Xmat)^+ \cvec}$.

First we generate $\hat{\sigma^2}$ from the residuals of the model.

Calculate the fitted values and the residuals given the $\bhat$ that you have
already.

In [8]:
#- Calculate the fitted and residual values
fitted = X @ B
residuals = Y - fitted
# Show residuals
residuals

In [9]:
assert len(residuals) == n
assert np.isclose(np.mean(residuals), 0)

We want an unbiased variance estimate for $\hat\sigma^2$.  See the [worked
example of GLM](https://textbook.nipraxis.org/mean_test_example.html) page and the [unbiased variance estimate](https://textbook.nipraxis.org/hypothesis_tests.html#unbiased-variance) section for
details.

The general rule is that we divide the sum of squares by $n - m$ where $m$ is
the number of *independent* columns in the design matrix.  Specifically, $m$
is the [matrix rank](http://matthew-brett.github.io/teaching/matrix_rank.html) of the design $\Xmat$.  $m$ can also be called the
*degrees of freedom of the design*.  $n - m$ is the *degrees of freedom of the
error* (see [unbiased variance estimate](https://textbook.nipraxis.org/hypothesis_tests.html#unbiased-variance)).

In [10]:
#- Calculate the degrees of freedom consumed by the design.
m = npl.matrix_rank(X)
#- Calculate the degrees of freedom of the error.
df_error = n - m
# Show degrees of freedom due to error.
df_error

Calculate the unbiased *variance* estimate $\hat{\sigma^2}$ by dividing the
sums of squares of the residuals by the degrees of freedom of the error.

In [11]:
#- Calculate the unbiased variance estimate
var_hat = np.sum(residuals ** 2) / df_error
# Show the result
var_hat

In [12]:
assert np.round(var_hat, 3) == 13.049

Now the calculate second part of the t statistic denominator,  $\cvec^T (\Xmat^T
\Xmat)^+ \cvec$. You already know that $\Xmat^T \Xmat$ is invertible, and you
have its inverse above, so you can use the inverse instead of the more general
pseudo-inverse.

In [13]:
#- Calculate c (X.T X)^-1 c.T
c_iXtX_ct = c @ npl.inv(X.T @ X) @ c.T
# Show the result
c_iXtX_ct

In [14]:
assert np.isclose(c_iXtX_ct, 0.4)

The final deminator of the t-statistic is the square root of the estimated variance `var_hat` mutiplied by the second half you have just calculated. The t-statistic is the numerator divided by the denominator.

In [15]:
#- Calculate t-statistic
t_stat = top_of_t / np.sqrt(var_hat * c_iXtX_ct)
# Show the result
t_stat

In [16]:
assert np.round(t_stat, 3) == 1.497

How likely is a t-statistic value this positive, or more positive, if there was in fact no underlying difference between the groups ? Use the `stats` module from `scipy` to create a
t-distribution with `df_error` (degrees of freedom of the error).  See the
`t_stat` function in [introduction to the general linear model](https://matthew-brett.github.io/teaching/glm_intro.html) for
inspiration:

In [17]:
#- Use scipy.stats to give a probability of a value as positive as the one you observe.
import scipy.stats as sst
tdistrib = sst.t(df_error)
# 1 - cumulative density function (P(x <= t)
1. - tdistrib.cdf(t_stat)
# This is the same as the "survival function"
p_val = tdistrib.sf(t_stat)
# Show the result
p_val

Check your result against the Scipy implementation of the independent t-test.

Scipy, by default, calculates the two-tailed p-value, whereas you have calculated the one-sided value.  Scipy's p value should be very close to 2 x your p-value.

In [18]:
t_test_result = sst.ttest_ind(mit_psycho, ucb_psycho)
t_test_result

In [19]:
assert np.isclose(t_test_result.statistic, t_stat)
assert np.isclose(t_test_result.pvalue, p_val * 2)

## Advanced hypothesis testing: F-tests

Imagine we have also measured the clammy score for the Berkeley and MIT
students.

In [20]:
#: Clamminess of handshake for UCB and MIT students
clammy = np.array([2.6386, 9.6094, 8.3379, 6.2871, 7.2775, 2.4787,
                   8.6037, 12.8713, 10.4906, 5.6766])

We want to test whether the clammy score is useful in explaining
the psychopathy data, over and above the students’ college affiliation.

To do this, we will use an [F test](https://textbook.nipraxis.org/hypothesis_tests.html#f-tests).

An F-test compares a *full model* $\Xmat_f$ with a *reduced model* $\Xmat_r$.

In our case, $\Xmat_f$ is the model containing the `clammy` regressor, as
well as the two dummy columns for the UCB and MIT group means.

$\Xmat_r$ is our original model, that only contains the dummy columns for the
UCB and MIT group means.

We define $SSR(\Xmat_r)$ and $SSR(\Xmat_f)$ as in [hypothesis tests](https://textbook.nipraxis.org/hypothesis_tests.html).
These are the Sums of Squares of the Residuals of the reduced and full model
respectively.

$$
\bhat_r = \Xmat_r^+ \yvec \\
\hat\evec_r = \yvec - \Xmat_r \bhat_r \\
SSR(\Xmat_r) = \hat\evec_r^T \hat\evec_r \\
\\
\bhat_f = \Xmat_f^+ \yvec \\
\hat\evec_f = \yvec - \Xmat_f \bhat_f \\
SSR(\Xmat_f) = \hat\evec_f^T \hat\evec_f
$$

You can calculate the F statistic for adding the `clammy` regressor, by
using these calculations and the formula for the F-test in [F tests](https://textbook.nipraxis.org/hypothesis_tests.html#f-tests).



























